# Galileo: LangGraph Agent with MongoDB Atlas Vector Search for RAG

This notebook demonstrates how to build a RAG (Retrieval Augmented Generation) agent using LangGraph, with MongoDB Atlas serving as the vector store. The agent will retrieve information from Lilian Weng's blog posts to answer questions.


**Key Components:**
1.  **Data Loading & Preprocessing:** Loads blog posts, splits them into chunks.
2.  **MongoDB Atlas Vector Store:** Initializes and populates a vector store with document embeddings.
3.  **Retriever:** Creates a retriever from the vector store.
4.  **LangGraph Agent:** Defines an agent that can use the retriever tool to answer questions.
5.  **Galileo Logging:** Integrates Galileo for observability.

## 1. Installation

Install the necessary Python packages. The original script used `%pip install`, which is suitable for notebooks.

In [ ]:
%pip install --upgrade --quiet galileo "pymongo[srv]" langchain-mongodb tiktoken langchain-google-genai langchain-openai langgraph langchain-community langchain-text-splitters -q

## 2. Setup Environment and API Keys

Import libraries and configure API keys. This notebook uses OpenAI for embeddings and the LLM, and a MongoDB Atlas URI for the vector store.

In [ ]:
import os
from uuid import uuid4

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore # Not used in final setup but present in original imports
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated, Sequence, TypedDict, Literal
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage
from langchain_core.prompts import PromptTemplate # Not explicitly used but good practice to import if planning prompts

from galileo.handlers.langchain import GalileoCallback
from galileo import GalileoLogger

# Configure OpenAI API Key
# If running in Google Colab, it attempts to load the key from Colab secrets.
# Otherwise, ensure the OPENAI_API_KEY environment variable is set.
if not os.getenv("OPENAI_API_KEY"):
  try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("OpenAI API Key loaded from Colab secrets.")
  except ImportError:
    print("Not in Colab environment. Please set the OPENAI_API_KEY environment variable.")

## 3. Load and Prepare Documents

Load content from specified URLs (Lilian Weng's blog posts) and then split them into smaller, manageable chunks for embedding and retrieval.

In [ ]:
urls = [
    "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

print(f"Loaded {len(docs_list)} documents.")
print(f"Split into {len(doc_splits)} chunks.")

## 4. Setup MongoDB Atlas Vector Store

Initialize the MongoDB client and define database, collection, and vector search index names. Then, set up the `MongoDBAtlasVectorSearch` instance.
For more info  [read more.](https://langchain-mongodb.readthedocs.io/en/latest/langchain_mongodb/vectorstores/langchain_mongodb.vectorstores.MongoDBAtlasVectorSearch.html#langchain_mongodb.vectorstores.MongoDBAtlasVectorSearch)



If your MongoDB Atlas cluster has IP whitelisting enabled, you might need to add the IP address of your current environment. The command below helps find your public IP. Add this IP to your Atlas cluster's Network Access list.
You can check your IP via `curl checkip.amazonaws.com`.

For `MongoDBAtlasVectorSearch` to work, a vector search index (named `ATLAS_VECTOR_SEARCH_INDEX_NAME` in this case) must exist in your MongoDB Atlas collection

In [ ]:

# MongoDB Atlas connection URI
# Replace with your actual MongoDB Atlas URI if different
uri = "mongodb+srv://<ATLAS-ACCOUNT>.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"
# Initialize MongoDB python client
client = MongoClient(uri)

DB_NAME = "langchain_test_db"
COLLECTION_NAME = "langchain_test_vectorstores"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "langchain-test-index-vectorstores"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

print(f"Using MongoDB Database: {DB_NAME}, Collection: {COLLECTION_NAME}")
print(f"Expected Atlas Vector Search Index: {ATLAS_VECTOR_SEARCH_INDEX_NAME}")
# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# Initialize MongoDBAtlasVectorSearch
vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine", # Optional: specifies the score function, cosine is common
)

vector_store.create_vector_search_index(dimensions=1536)

print("MongoDBAtlasVectorSearch initialized.")

## 5. Populate Vector Store and Create Retriever

Add the processed document chunks to the MongoDB Atlas vector store and then create a retriever interface for querying.

In [ ]:
# Generate unique IDs for each document chunk
uuids = [str(uuid4()) for _ in range(len(doc_splits))]

# Add documents to the vector store
# This will embed the documents and store them in MongoDB Atlas.
# Ensure your MongoDB collection is empty or you manage IDs appropriately if re-running.
print(f"Adding {len(doc_splits)} document chunks to MongoDB Atlas...")
try:
    # For a clean run, you might want to delete existing documents if re-populating the same collection.
    # MONGODB_COLLECTION.delete_many({})
    # print("Cleared existing documents from the collection.")
    vector_store.add_documents(documents=doc_splits, ids=uuids)
    print("Documents added successfully.")
except Exception as e:
    print(f"Error adding documents: {e}")
    print("This might be due to issues with index existence/configuration or network access.")

# Create a retriever from the vector store
retriever = vector_store.as_retriever()
print("Retriever created.")

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts.",
)

tools = [retriever_tool]

print("Retriever tool created.")

# Test the retriever tool (optional)
print("\nTesting retriever tool...")
try:
    test_retrieval = retriever_tool.invoke({"query": "types of reward hacking"})
    print("Retriever tool test output:", test_retrieval)
except Exception as e:
    print(f"Error testing retriever tool: {e}")

## 6. Build the LangGraph Agent

Define the agent's state, the core agent logic (which decides whether to use tools or respond directly), and construct the graph.

In [ ]:
class AgentState(TypedDict):
  # The add_messages function defines how an update should be processed
  # Default is to replace. add_messages says "append"
  messages: Annotated[Sequence[BaseMessage], add_messages]

def agent(state):
  """
  Invokes the agent model to generate a response based on the current state. Given
  the question, it will decide to retrieve using the retriever tool, or simply end.

  Args:
      state (messages): The current state

  Returns:
      dict: The updated state with the agent response appended to messages
  """
  print("--- CALLING AGENT ---")
  messages = state["messages"]
  # Using gpt-4-turbo as it's generally better at tool use
  model = ChatOpenAI(temperature=0, streaming=True, model="gpt-4-turbo-preview") # or "gpt-3.5-turbo"
  model = model.bind_tools(tools)
  response = model.invoke(messages)
  # We return a list, because this will get added to the existing list
  return {"messages": [response]}

retrieve_node = ToolNode([retriever_tool])

# Define the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", agent)
workflow.add_node("retrieve", retrieve_node)

# Define edges
workflow.add_edge(START, "agent") # Start with the agent

workflow.add_conditional_edges(
    "agent",
    tools_condition, # LangGraph's built-in function to check for tool calls
    {
        # If the agent decided to call a tool, execute the retrieve node
        "tools": "retrieve",
        # Otherwise, END the workflow
        END: END,
    },
)

workflow.add_edge("retrieve", "agent") # After retrieval, go back to the agent to process results

# Compile the graph
graph = workflow.compile()
print("LangGraph agent compiled.")

## 7. Setup Galileo Logging ✨

Configure environment variables for Galileo and initialize the Galileo callback handler for Langchain.

🔗 See full documentation [here](https://v2docs.galileo.ai/references/faqs/faqs#q%3A-how-do-i-authenticate-with-the-galileo-api%3F)



In [ ]:
# Replace with your Galileo API Key, Project Name, and desired Log Stream
os.environ["GALILEO_API_KEY"] = "<YOUR_GALILEO_API_KEY>" # Replace with your key
os.environ["GALILEO_PROJECT"]= "elastic" # Replace with your project name
os.environ["GALILEO_LOG_STREAM"] = "my_log_stream" # Replace with your stream name

galileo_handler = GalileoCallback()

print("Galileo callback handler initialized.")
print(f"GALILEO_API_KEY set: {'GALILEO_API_KEY' in os.environ}")
print(f"GALILEO_PROJECT: {os.getenv('GALILEO_PROJECT')}")
print(f"GALILEO_LOG_STREAM: {os.getenv('GALILEO_LOG_STREAM')}")

## 8. Run the Agent

Define an input query and invoke the compiled LangGraph agent. The Galileo callback will capture traces.

In [ ]:
inputs = {
    "messages": [
        HumanMessage(content="What does Lilian Weng say about the types of agent memory?"),
    ]
}

print(f"Invoking agent with query: {inputs['messages'][0].content}")

try:
    # The recursion_limit is important for agents that might loop.
    # Callbacks list includes the Galileo handler.
    final_state = graph.invoke(inputs, {"recursion_limit": 5, "callbacks": [galileo_handler]})

    print("\n--- FINAL RESPONSE ---")
    if final_state and 'messages' in final_state and final_state['messages']:
        # The final response is usually the last message from the AI
        print(final_state['messages'][-1].content)
    else:
        print("No final message content found in the state.")
        print("Final state:", final_state)

except Exception as e:
    print(f"Error during graph invocation: {e}")
    print("This could be due to API key issues, network problems, or errors in the agent/tool logic.")